# Прибыльные профили приложений для App Store и Google Play Markets

## Описание проблемы

Компания занимающаяся разработкой условно-бесплатных мобильных приложений для IOS и Android, получающих доход за счет рекламы (Adware), имеет цель достичь большего количества использования этих приложений среди пользователей.

Чтобы добиться этого, необходимо сделать предварительный анализ рынка приложений на предмет популярности  различных приложений среди разных групп пользователей. Ориентировавшись на полученные данные, компания сможет вырабатать успешную стратегию, акцентируя усилия на более перспективных приложениях.
Сделать такой анализ можно воспользовавшись находящимися в открытом доступе данными с площадок App Store и Google Markets.

## Цель проекта

Помочь разработчикам определить, какой тип мобилных приложений будет наиболее востребованным среди пользователей

## Открытие файла и первоначальное ознакомление с данными 

По состоянию на первый квартал 2020 года площадки Apple App Store и Google Play Market [являются наиболее популярными среди пользователей](https://www.statista.com/statistics/276623/number-of-apps-available-in-leading-app-stores/).
![](apps_chart.png)
Исходя из этого, для анализа были выбраны соответствующие наборы данных:
* [Google Play Store Apps](https://www.kaggle.com/lava18/google-play-store-apps) содержит около 10 тысяч приложений, может быть скачан [здесь](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv)
* [Mobile App Store](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/metadata) содержит около 7 тысяч приложений, может быть скачан [здесь](https://dq-content.s3.amazonaws.com/350/AppleStore.csv)

Для начала работы с наборами данных необходимо конвертировать CSV файлы с данными в списки.
Создадим для этого функцию `csv_to_list()`, принимающую имя файла в качестве значения аргумента. Присвоим полученные списки переменным `apple_store_data` и `google_play_data`.

In [1]:
def csv_to_list(csv_file_name):
    from csv import reader
    opened_file = open(csv_file_name)
    read_file = reader(opened_file)
    apps_data = list(read_file)
    opened_file.close
    return apps_data

### Apple Store Data Set ###
apple_store_data = csv_to_list('AppleStore.csv')

### Google Play Data Set ###
google_play_data = csv_to_list('googleplaystore.csv')

Для облегчения ознакомления с наборами данных создадим функцию `explore_data()`, которая будет выводить в читабельном виде срез значений из указанного списка в заданном диапазоне и количество рядов и колонок в указанном списке.

In [2]:
def explore_data(dataset, start, end, rows_and_columns = False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
    if rows_and_columns:
        print('Number of rows: ', len(dataset) - 1)
        print('Number of columns: ', len(dataset[0]))

In [3]:
# Выводим первые 3 ряда из датасета Google Play
explore_data(google_play_data, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows:  10841
Number of columns:  13


Набор данных Google Play Market содержит 10841 ряд и 13 колонок. Для целей нашего исследования будут полезны
следующие колонки: 'App', 'Category', 'Rating', 'Reviews', 'Installs', 'Type', 'Price', 'Genres' ('App', 'Category', 'Reviews', 'Installs', 'Type', 'Price', and 'Genres').

In [4]:
# Выводим первые 3 ряда из датасета Apple Store
explore_data(apple_store_data, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows:  7197
Number of columns:  16


Набор данных Apple App Store содержит 7197 рядов и 16 колонок. Для нашего исследования могут быть полезны следующие колонки: 'track_name', 'price', 'rating_count_tot', 'user_rating', prime_genre ('track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', and 'prime_genre'). Не все названия колонок являются понятными, поэтому для выяснения их значений можно обратиться к [документации](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) этого набора данных.

## Удаление ненужных данных

Выше мы поверхностно проанализировали данные с которыми предстоит работа. Для выполнения полноценного анализа мы должны убедиться в том, что имеющиеся наборы данных будут содержать только корректную информацию, анализ которой не вызовет ошибок в наших расчетах.

Очистка данных будет включать в себя следующие операции:
* исправление записей содержащих некорректную информацию или удаление их
* удаление дублирующихся записей
* удаление всех записей о неанглоязычных приложениях (текущая задача подразумивает анализ только англоязычной аудитории)
* удаление всех платных приложений (наша задача подразумевает анализ только беспатных приложений)

Создадим функцию `clean_another_length_rows()` которая будет получать список, содержащий заголовки в первой строке 
и возвращающую новый список, где будут удалены все строки, колличество элементов в которых будет отличаться от колличества элементов в строке заголовков

In [5]:
def clean_another_length_rows(datalist, info = False):
    headers_length = len(datalist[0])
    origin_length = len(datalist)
    for index, row in enumerate(datalist):
        if len(row) != headers_length:
            if info:
                print('row number: ', index)
                print(row, '\n')
            del datalist[index]        
    if info:
        print('origin length: ', origin_length)
        print('cleaned length: ', len(datalist))        
    return datalist

С помощью функции `clean_another_length_rows()` очистим список с данными Google Play Market `google_play_data`:

In [6]:
google_play_data = clean_another_length_rows(google_play_data, info = True)

row number:  10473
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up'] 

origin length:  10842
cleaned length:  10841


В результате очистки мы удалили одну строку 10473, содержащую некорректные данные

С помощью функции `clean_another_length_rows()` очистим список с данными Apple Store `apple_store_data`:

In [7]:
apple_store_data = clean_another_length_rows(apple_store_data, info = True)

origin length:  7198
cleaned length:  7198


В результате очистки `apple_store_data` функцией `clean_another_length_rows()` некорректных данный обнаружено
не было, поэтому список `apple_store_data`остается неизменным.

In [8]:
## Выводы